In [246]:
import pandas as pd
import csv
import re
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score


In [247]:
filePath23 = r'Movies2023.csv'
filePath24 = r'movies2024.csv'

In [248]:
df1 = pd.read_csv(filePath23)
df2 = pd.read_csv(filePath24)
df = pd.concat([df1,df2],axis = 0)
df.to_csv('combinedData.csv',index=False)
df.head()

,Unnamed: 0,@type,url,name,description,image,contentRating,genre,duration,Language,...,aggregateRating.worstRating,aggregateRating.ratingValue,aggregateRating.ratingCount,alternateName,First Day Collection India,India Net Collection,India Gross collection,India Net collection,Note,OLanguage
0,0,Movie,https://www.imdb.com/title/tt15354916/,Jawan,A prison warden recruits inmates to commit out...,https://m.media-amazon.com/images/M/MV5BOWI5Nm...,Not Rated,"Action, Crime, Drama",PT2H49M,Hindi,...,1,6.9,99934,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Movie,https://www.imdb.com/title/tt12844910/,Pathaan,An Indian agent races against a doomsday clock...,https://m.media-amazon.com/images/M/MV5BYTgzNj...,Not Rated,"Action, Adventure, Thriller",PT2H26M,Hindi,...,1,5.8,158261,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Movie,https://www.imdb.com/title/tt13751694/,Animal,The hardened son of a powerful industrialist r...,https://m.media-amazon.com/images/M/MV5BNGViM2...,Not Rated,"Action, Crime, Drama",PT3H24M,Hindi,...,1,6.1,98028,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Movie,https://www.imdb.com/title/tt15441054/,Gadar 2,When Tara Singh goes missing during a skirmish...,https://m.media-amazon.com/images/M/MV5BODJkYj...,Not Rated,"Action, Adventure, Drama",PT2H50M,Hindi,...,1,5.1,22652,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Movie,https://www.imdb.com/title/tt13927994/,Salaar: Part 1 - Ceasefire,The fate of a violently contested kingdom hang...,https://m.media-amazon.com/images/M/MV5BYWM5OG...,Not Rated,"Action, Crime, Drama",PT2H55M,Telugu,...,1,6.6,69295,Salaar,NaN,NaN,NaN,NaN,NaN,NaN


DATA CLEANING

In [249]:
def dataCleaning(df):
    df = df.drop(columns=['aggregateRating.@type', 'aggregateRating.bestRating','aggregateRating.worstRating','Unnamed: 0','@type'])
    df = df.rename(columns = {'aggregateRating.ratingValue':'ratingValue','aggregateRating.ratingCount':'ratingCount'})
    df = df[df['Language'] != 'English']
    df['Budget'] = df['Budget'].apply(lambda x: float(x.replace('cr','')))
    df['First Day Collection Worldwide'] = df['First Day Collection Worldwide'].apply(lambda x: float(x.replace('cr','')))
    df['Worldwide Collection'] = df['Worldwide Collection'].apply(lambda x: float(x.replace('cr','')))
    df = df.loc[:,:'Verdict']
    concat_genre = df['genre'].str.cat(sep=', ')
    genre_set = (set(concat_genre.split(', ')))
    for colName in genre_set:
        df[colName]=0
    row=-1
    for genre_type in df['genre']:
        row+=1
        genre_sep = genre_type.split(', ')
        for val in genre_sep:
            df.loc[row,val]=1
    df = df.iloc[:145,:]
    categories = df['Verdict'].astype('category').cat.categories
    code_to_category = pd.DataFrame({'Code': range(len(categories)),'Verdict': categories})
    df['Verdict'] = df['Verdict'].astype('category').cat.codes
    print(code_to_category)
    
    categories = df['Language'].astype('category').cat.categories
    code_to_category = pd.DataFrame({'Code': range(len(categories)),'Language': categories})
    print(code_to_category)
    
    df['Language'] = df['Language'].astype('category').cat.codes
    df = df.drop(columns=['url', 'name', 'description', 'image', 'contentRating', 'genre','duration','Overseas Collection', 'India Gross Collection'])

    return df

In [250]:
df = dataCleaning(df)
df.to_csv('finalDf.csv',index=False)

    Code                Verdict
0      0          Above Average
1      1   All Time Blockbuster
2      2                Average
3      3          Below Average
4      4            Blockbuster
5      5  Blockbuster as of now
6      6               Disaster
7      7                   Flop
8      8                    Hit
9      9      Pending as of now
10    10              Super Hit
11    11    Super Hit as of now
   Code   Language
0     0      Hindi
1     1    Kannada
2     2  Malayalam
3     3    Marathi
4     4    Punjabi
5     5      Tamil
6     6     Telugu


In [251]:
X = df.drop(columns=['Worldwide Collection'])
y = df[['Worldwide Collection']]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
# lr = LinearRegression()
# lr.fit(X_train,y_train)

In [252]:
X

,Language,Budget,First Day Collection Worldwide,Verdict,Family,Musical,Fantasy,Drama,Adventure,Biography,...,Crime,Mystery,Romance,Action,Thriller,Horror,Comedy,War,Sci-Fi,Sport
0,0,285.0,129.6,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,250.0,104.8,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0,140.0,115.9,4,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,80.0,53.7,4,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,250.0,165.3,8,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,50.0,4.2,6,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
71,2,15.0,1.8,7,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
72,6,8.0,1.8,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
73,6,20.0,4.3,7,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [253]:
def linear_reg_model_movie(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    lr = LinearRegression()
    lr.fit(X_train,y_train)
    y_pred_train = lr.predict(X_train)
    y_pred_test = lr.predict(X_test)

    return lr,X_train, X_test, y_train, y_test,y_pred_train,y_pred_test

In [254]:
lr,X_train, X_test, y_train, y_test,y_pred_train,y_pred_test = linear_reg_model_movie(X,y)


In [255]:
X_train.head()

,Language,Budget,First Day Collection Worldwide,Verdict,Family,Musical,Fantasy,Drama,Adventure,Biography,...,Crime,Mystery,Romance,Action,Thriller,Horror,Comedy,War,Sci-Fi,Sport
32,5,80.0,24.2,7,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
45,6,30.0,7.8,8,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
18,0,125.0,26.3,2,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
46,5,30.0,11.2,4,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
65,5,15.0,1.6,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [256]:
y_train.head()

,Worldwide Collection
32,69.8
45,37.8
18,182.5
46,73.8
65,43.5


In [257]:
y_pred_train

array([[ 7.33760108e+01],
       [ 1.63988522e+01],
       [ 2.55013714e+02],
       [ 5.80114654e+01],
       [-4.44660614e-01],
       [ 8.45147186e+02],
       [ 2.74647725e+01],
       [ 1.94452010e+02],
       [-2.79106492e+01],
       [ 1.40924667e+02],
       [ 1.56800410e+02],
       [ 1.48213286e+02],
       [ 1.98563351e+01],
       [ 8.84780533e+01],
       [ 2.20173935e+01],
       [ 7.65159594e+01],
       [ 5.53067400e+01],
       [ 4.03506526e+01],
       [-7.50894702e+00],
       [ 1.81803677e+02],
       [ 1.11117863e+02],
       [ 1.47169330e+02],
       [ 4.30342256e+02],
       [ 1.29490740e+01],
       [ 3.21651501e+01],
       [ 1.92889881e+02],
       [ 5.15664005e+01],
       [ 1.97160644e+01],
       [ 4.00150889e+01],
       [ 1.39262109e+02],
       [ 1.96338213e+02],
       [-4.61899175e+00],
       [ 5.94109306e+01],
       [ 1.68911310e+02],
       [ 8.66447663e+00],
       [ 4.58873099e+01],
       [ 1.01291121e+02],
       [ 1.15530022e+02],
       [ 4.8

In [239]:
r2_score1 = r2_score(y_pred_train,y_train)
r2_score1

0.8238622868634551

In [240]:
r2_score1 = r2_score(y_pred_test,y_test)
r2_score1

0.7313247395061019

In [241]:
# testFile = r'/Users/vineetjujjavarapu/Desktop/gitrepos/movieTestNew.csv'
testFile = r'/Users/vineetjujjavarapu/Desktop/gitrepos/MovieGrossCollection/MoviePredTestData.csv'

In [242]:
testdf = pd.read_csv(testFile)
testdf

,Language,Budget,First Day Collection Worldwide,Worldwide Collection,Verdict,Comedy,Crime,Family,Sport,Sci-Fi,...,Drama,Horror,Action,Adventure,War,Musical,Biography,Mystery,Romance,History
0,0,285.0,129.6,1150.7,1,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0
1,6,285.0,150.0,803.0,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,125.0,73.2,415.0,6,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,35.0,23.4,159.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0
4,6,450.0,182.6,886.0,4,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1,1.0,1.0,0.0,0.0,0.0,0.0
5,0,240.0,37.8,252.0,2,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,285.0,150.0,803.0,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0
7,6,285.0,150.0,803.0,5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0
8,6,285.0,150.0,803.0,6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,450.0,182.6,886.0,4,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1,1.0,1.0,0.0,0.0,0.0,0.0


In [243]:
X = testdf.drop(columns=['Worldwide Collection'])
y = testdf[['Worldwide Collection']]

In [244]:
testdf.columns

Index(['Language', 'Budget', 'First Day Collection Worldwide',
       'Worldwide Collection', 'Verdict', 'Comedy', 'Crime', 'Family', 'Sport',
       'Sci-Fi', 'Fantasy', 'Thriller', 'Drama', 'Horror', 'Action',
       'Adventure', 'War', 'Musical', 'Biography', 'Mystery', 'Romance',
       'History'],
      dtype='object')

In [245]:
y_new_test= lr.predict(X)
y_new_test

ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.


In [ ]:
from sklearn.linear_model import Ridge 
from sklearn.model_selection import GridSearchCV

ridge = Ridge()

params={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,100]}
ridge_regressor = GridSearchCV(ridge,params,scoring = 'neg_mean_absolute_error',cv = 5)
ridge_regressor.fit(X_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.10841e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.09418e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.33681e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.28436e-21): result may not be acc

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 100]},
             scoring='neg_mean_absolute_error')

In [ ]:
from sklearn.linear_model import Lasso 
from sklearn.model_selection import GridSearchCV

lasso = Lasso()

params={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,100]}
lasso_regressor = GridSearchCV(lasso,params,scoring = 'neg_mean_absolute_error',cv = 5)
lasso_regressor.fit(X_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.702e+05, tolerance: 4.846e+02
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.522e+05, tolerance: 5.007e+02
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 100]},
             scoring='neg_mean_absolute_error')

In [ ]:
ridge_pred_train = ridge_regressor.predict(X_train)
lasso_pred_train = lasso_regressor.predict(X_train)
ridge_pred_test = ridge_regressor.predict(X_test)
lasso_pred_test = lasso_regressor.predict(X_test)

print(f"r2score for ridge_train = {r2_score(ridge_pred_train,y_train)}")
print(f"r2score for lasso_train = {r2_score(lasso_pred_train,y_train)}")
print(f"r2score for ridge_test = {r2_score(ridge_pred_test,y_test)}")
print(f"r2score for ridge_test = {r2_score(lasso_pred_test,y_test)}")

r2score for ridge_train = 0.8028052450668424
r2score for lasso_train = 0.7681056396599202
r2score for ridge_test = 0.7886701072781132
r2score for ridge_test = 0.7870622201397106


In [ ]:
ridge_pred_test_new = ridge_regressor.predict(X)
lasso_pred_test_new = lasso_regressor.predict(X)

ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.


In [ ]:
ridge_pred_test_new

NameError: name 'ridge_pred_test_new' is not defined

In [ ]:
lasso_pred_test_new

NameError: name 'lasso_pred_test_new' is not defined

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

knn_regressor = KNeighborsRegressor(n_neighbors=18)  # Set the number of neighbors (k)
knn_regressor.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = knn_regressor.predict(X_test)

# Step 5: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

Mean Squared Error: 3588.05


In [ ]:
r2_score(y_test,y_pred)

NameError: name 'y_pred' is not defined

In [ ]:
y_pred_new = knn_regressor.predict(X)
y_pred_new

array([[491.84444444],
       [491.84444444],
       [274.15555556],
       [ 80.96944444],
       [491.84444444],
       [427.11111111],
       [491.84444444],
       [491.84444444],
       [491.84444444],
       [491.84444444]])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.datasets import make_regression

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Train the Random Forest Regressor model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)  # Using 100 trees in the forest
rf_regressor.fit(X_train, y_train)

# Step 5: Make predictions
y_pred = rf_regressor.predict(X_test)

# Step 6: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
mse

np.float64(56856.19681633322)

In [ ]:
y_pred_new = rf_regressor.predict(X)

y_pred_new

array([1005.783,  825.722,  400.161,  340.663,  883.264,  664.301,
        847.571,  822.402,  795.242,  883.264])

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
base_estimator = DecisionTreeRegressor(max_depth=4)
ada_regressor = AdaBoostRegressor(n_estimators=100, random_state=42)
ada_regressor.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(n_estimators=100, random_state=42)

In [ ]:
ada_regressor.fit(X_train, y_train)
y_pred = ada_regressor.predict(X_test)

# Step 6: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NameError: name 'mean_squared_error' is not defined

In [ ]:
y_pred_new = ada_regressor.predict(X)
y_pred_new

array([1150.7,  803. ,  415. ,  159. ,  886. ,  415. ,  803. ,  803. ,
        803. ,  886. ])

In [ ]:
X_train

,Language,Budget,First Day Collection Worldwide,Verdict,Crime,Thriller,Family,War,Fantasy,Biography,...,Mystery,Drama,Sport,Adventure,Sci-Fi,History,Musical,Comedy,Horror,Action
32,5,80.0,24.2,7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
45,6,30.0,7.8,8,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
18,0,125.0,26.3,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
46,5,30.0,11.2,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
65,5,15.0,1.6,4,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,6,375.0,136.8,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
31,2,40.0,15.8,8,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
78,6,8.0,1.8,4,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
46,5,5.0,1.8,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
X = df.drop(columns=['Worldwide Collection'])
y = df[['Worldwide Collection']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

In [ ]:
X_train['Verdict'] = X_train['Verdict']*10
X_train

,Language,Budget,First Day Collection Worldwide,Verdict,Family,Musical,Fantasy,Drama,Adventure,Biography,...,Crime,Mystery,Romance,Action,Thriller,Horror,Comedy,War,Sci-Fi,Sport
32,5,80.0,24.2,70,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
45,6,30.0,7.8,80,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
18,0,125.0,26.3,20,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
46,5,30.0,11.2,40,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
65,5,15.0,1.6,40,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,6,375.0,136.8,20,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
31,2,40.0,15.8,80,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
78,6,8.0,1.8,40,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
46,5,5.0,1.8,50,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
lr1 = LinearRegression()
weights = X_train['Verdict']*10
lr1.fit(X_train_scaled,y_train,sample_weight=X_train['Verdict'])
y_pred_train = lr1.predict(X_train_scaled)
y_pred_test = lr1.predict(X_test_scaled)

In [ ]:
r2_score(y_train,y_pred_train)

0.838098409279193

In [ ]:
r2_score(y_test,y_pred_test)

0.6776823880952104

In [ ]:
X_train.columns
for idx, column in enumerate(df.columns):
    print(f"Index: {idx}, Column: {column}")

Index: 0, Column: Language
Index: 1, Column: Budget
Index: 2, Column: First Day Collection Worldwide
Index: 3, Column: Worldwide Collection
Index: 4, Column: Verdict
Index: 5, Column: Family
Index: 6, Column: Musical
Index: 7, Column: Fantasy
Index: 8, Column: Drama
Index: 9, Column: Adventure
Index: 10, Column: Biography
Index: 11, Column: History
Index: 12, Column: Crime
Index: 13, Column: Mystery
Index: 14, Column: Romance
Index: 15, Column: Action
Index: 16, Column: Thriller
Index: 17, Column: Horror
Index: 18, Column: Comedy
Index: 19, Column: War
Index: 20, Column: Sci-Fi
Index: 21, Column: Sport


In [ ]:
data = [
    [6, 125,73.5, 6, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
]
forecasting_df = pd.DataFrame(data,columns=X_train.columns)
forecasting_df

,Language,Budget,First Day Collection Worldwide,Verdict,Family,Musical,Fantasy,Drama,Adventure,Biography,...,Crime,Mystery,Romance,Action,Thriller,Horror,Comedy,War,Sci-Fi,Sport
0,6,125,73.5,6,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
forecasting_df_scaled = scaler_X.transform(forecasting_df)

In [ ]:
y_pred_forecating = lr1.predict(forecasting_df_scaled)
y_pred_forecating

array([[358.22499486]])

In [ ]:
from sklearn.preprocessing import RobustScaler
import pandas as pd

scalar = RobustScaler()
df_robust_scalar_X_train = pd.DataFrame(scalar.fit_transform(X_train))
df_robust_scalar_X_test = scalar.transform(X_test)

In [ ]:
lr2 = LinearRegression()
lr2.fit(df_robust_scalar_X_train,y_train)
y_pred_train = lr2.predict(df_robust_scalar_X_train)
y_pred_test = lr2.predict(df_robust_scalar_X_test)

In [ ]:
r2_score(y_pred_train,y_train)

0.8238622868634545

In [ ]:
r2_score(y_pred_test,y_test)

0.7313247395061018

In [ ]:
forecasting_df_scaled = scalar.transform(forecasting_df)
y_pred_forecating_robust = lr2.predict(forecasting_df_scaled)
y_pred_forecating_robust

array([[370.55701601]])

RIDGE WITH SCALAR

In [ ]:
from sklearn.linear_model import Ridge 
from sklearn.model_selection import GridSearchCV

ridge = Ridge()

params={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,100]}
ridge_regressor = GridSearchCV(ridge,params,scoring = 'neg_mean_absolute_error',cv = 5)
ridge_regressor.fit(df_robust_scalar_X_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.77874e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.30005e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.09811e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.81057e-19): result may not be acc

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 100]},
             scoring='neg_mean_absolute_error')

In [ ]:
y_pred_train = ridge_regressor.predict(df_robust_scalar_X_train)
y_pred_test = ridge_regressor.predict(df_robust_scalar_X_test)

In [ ]:
r2_score(y_pred_train,y_train)

0.7519406925051356

In [ ]:
r2_score(y_pred_test,y_test)

0.8178900272346025

In [ ]:
forecasting_df_scaled = scalar.transform(forecasting_df)
y_pred_forecating_robust = ridge_regressor.predict(forecasting_df_scaled)
y_pred_forecating_robust

array([[389.67665372]])

MIN_MAX_SCALAR

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler_min_max = MinMaxScaler(feature_range=(-1,1))
min_max_scaled_df_X_train = scaler_min_max.fit_transform(X_train)
min_max_scaled_df_X_test = scaler_min_max.transform(X_test)


In [ ]:
lr3 = LinearRegression()
lr3.fit(min_max_scaled_df_X_train,y_train)
y_pred_train = lr3.predict(min_max_scaled_df_X_train)
y_pred_test = lr3.predict(min_max_scaled_df_X_test)
print(f'r2 for train {r2_score(y_pred_train,y_train)}')
print(f'r2 for test {r2_score(y_pred_test,y_test)}')

r2 for train 0.8238622868634549
r2 for test 0.731324739506102


GAUSSIAN TRANSFORMATIONS– POWER TRANSFORMER 

The Yeo-Johnson transformation is a data transformation technique that remaps each feature value such that the feature distribution is near-Gaussian, thus improving the predictive performance of ML algorithms that assume normality, such as linear regression, ElasticNet, and Bayesian regression.

In [ ]:
from sklearn.preprocessing import PowerTransformer
scaler_pt = PowerTransformer(method='yeo-johnson')

transformed_df_X_train = pd.DataFrame(scaler_pt.fit_transform(X_train), columns=X_train.columns)
transformed_df_X_test = pd.DataFrame(scaler_pt.transform(X_test), columns=X_train.columns)

In [ ]:
from sklearn.metrics import mean_squared_error
lr4 = LinearRegression()
lr4.fit(transformed_df_X_train,y_train)
y_pred_train = lr4.predict(transformed_df_X_train)
y_pred_test = lr4.predict(transformed_df_X_test)
print(f'r2 for train {r2_score(y_pred_train,y_train)}')
print(f'r2 for test {r2_score(y_pred_test,y_test)}')
print(f'mse is {mean_squared_error(y_test, y_pred)}')

r2 for train 0.31173288211686667
r2 for test 0.6338474027444692
mse is 17572.64855473269
